In [1689]:
import pandas as pd
import sklearn
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from numpy import reshape
from sklearn import metrics
from collections import Counter 

In [1690]:
# data = pd.read_csv('train_dec08_task3_copy.csv')
data = pd.read_csv('train_dec08_task3.csv')

In [1691]:
# split train and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=35)

In [1692]:
# split features and labels
X_train = train_data.iloc[:, :14]
y_train = train_data.iloc[:, 14]

X_test = test_data.iloc[:, :14]
y_test = test_data.iloc[:, 14]

# pred_data = pd.read_csv('test_dec08_task3_only_features_copy.csv')
pred_data = pd.read_csv('test_dec08_task3_only_features.csv')

In [1693]:
# tranform the labels to numbers to fit XGBoost
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [1694]:
# check the imbalance of the data
Counter(y_train)

Counter({1: 1606, 3: 778, 2: 2030, 0: 786})

In [1695]:
# use SMOTE to balance the data
# deprecated: SMOTE + TomekLinks is not better than SMOTE
from imblearn.over_sampling import SMOTE
X_train, y_train = SMOTE(random_state=42).fit_resample(X_train, y_train)

# check the imbalance of the data again
print(f"Training target statistics: {Counter(y_train)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({1: 2030, 3: 2030, 2: 2030, 0: 2030})
Testing target statistics: Counter({2: 470, 1: 394, 3: 222, 0: 214})


In [1696]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

# choose the best k features numbers
select_k = 5

selection = SelectKBest(mutual_info_classif, k=select_k).fit(X_train, y_train)

# show the selected features
''' Higher score means more important with the target 
    Lower score means less important with the target
'''

features = X_train.columns[selection.get_support()]
print(f"features: {features}")
print(f"scores: {selection.scores_}")
X_train = selection.fit_transform(X_train, y_train)
X_test = selection.transform(X_test)


features: Index(['feature2', 'feature3', 'feature4', 'feature5', 'feature6'], dtype='object')
scores: [0.         0.00841154 0.25755802 0.26769285 0.21908675 0.1618421
 0.15057226 0.         0.00333293 0.01802162 0.02377713 0.
 0.01193569 0.00128441]


In [1697]:
# build XGBClassifier model
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.36)

# train data with train_data
# xgboostModel.fit(X_train, y_train)
xgboostModel.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.36, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto', ...)

In [1698]:
# print the accuracy
print('訓練集: ',xgboostModel.score(X_train,y_train))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  0.9769704433497537
測試集:  0.7415384615384616


In [1699]:
# predict the real given data
pred_data = selection.transform(pred_data)
answer = xgboostModel.predict(pred_data)
answer = le.inverse_transform(answer)

In [1700]:
da = {"Id": [x for x in range(1, len(answer)+1)], "Category": answer}
df = pd.DataFrame (da)

In [1701]:
df.to_csv("output.csv", index=False)